# Intro to Recommender Systems Lab

Complete the exercises below to solidify your knowledge and understanding of recommender systems.

For this lab, we are going to be putting together a user similarity based recommender system in a step-by-step fashion. Our data set contains customer grocery purchases, and we will use similar purchase behavior to inform our recommender system. Our recommender system will generate 5 recommendations for each customer based on the purchases they have made.

In [1]:
import pandas as pd
from scipy.spatial.distance import pdist, squareform

In [2]:
data = pd.read_csv('./data/customer_product_sales.csv')

In [3]:
data.head()

,CustomerID,FirstName,LastName,SalesID,ProductID,ProductName,Quantity
0,61288,Rosa,Andersen,134196,229,Bread - Hot Dog Buns,16
1,77352,Myron,Murray,6167892,229,Bread - Hot Dog Buns,20
2,40094,Susan,Stevenson,5970885,229,Bread - Hot Dog Buns,11
3,23548,Tricia,Vincent,6426954,229,Bread - Hot Dog Buns,6
4,78981,Scott,Burch,819094,229,Bread - Hot Dog Buns,20


## Step 1: Create a data frame that contains the total quantity of each product purchased by each customer.

You will need to group by CustomerID and ProductName and then sum the Quantity field.

In [4]:
quantity_df = data.groupby(['CustomerID', 'ProductName']).sum()['Quantity'].reset_index()
quantity_df.head()

,CustomerID,ProductName,Quantity
0,33,Apricots - Dried,1
1,33,Assorted Desserts,1
2,33,Bandage - Flexible Neon,1
3,33,"Bar Mix - Pina Colada, 355 Ml",1
4,33,"Beans - Kidney, Canned",1


## Step 2: Use the `pivot_table` method to create a product by customer matrix.

The rows of the matrix should represent the products, the columns should represent the customers, and the values should be the quantities of each product purchased by each customer. You will also need to replace nulls with zeros, which you can do using the `fillna` method.

In [5]:
pivot_df = pd.pivot_table(quantity_df, columns='CustomerID', index='ProductName', values='Quantity', fill_value=0)
pivot_df.head()

CustomerID,33,200,264,356,412,464,477,639,649,669,...,97697,97753,97769,97793,97900,97928,98069,98159,98185,98200
ProductName,,,,,,,,,,,,,,,,,,,,,
Anchovy Paste - 56 G Tube,0,0,0,0,0,0,0,1,0,0,...,0,25,0,0,0,0,0,0,0,0
"Appetizer - Mini Egg Roll, Shrimp",0,0,0,0,0,0,0,0,0,0,...,25,25,0,0,0,0,0,0,0,0
Appetizer - Mushroom Tart,0,0,0,0,0,0,0,1,0,0,...,25,0,0,0,0,0,0,0,25,0
Appetizer - Sausage Rolls,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,25,25,25,0,25,0
Apricots - Dried,1,0,0,0,1,0,0,0,0,0,...,0,25,0,0,0,0,0,0,0,0


## Step 3: Create a customer similarity matrix using `squareform` and `pdist`. For the distance metric, choose "euclidean."

In [6]:
# Find distances between all users - Small distance between similar customers
distances = squareform(pdist(pivot_df.T, 'euclidean'))

In [7]:
# Dataframe + Conversion - Big distance between similar customers (if equal, then 1)
distances_df = pd.DataFrame(1 / (1 + distances), index=pivot_df.columns, columns=pivot_df.columns)
distances_df.head()

CustomerID,33,200,264,356,412,464,477,639,649,669,...,97697,97753,97769,97793,97900,97928,98069,98159,98185,98200
CustomerID,,,,,,,,,,,,,,,,,,,,,
33,1.000000,0.077421,0.087047,0.081800,0.080634,0.082709,0.074573,0.083020,0.081503,0.080070,...,0.004811,0.004669,0.004412,0.005019,0.004312,0.004515,0.004583,0.004355,0.004167,0.004333
200,0.077421,1.000000,0.078448,0.076435,0.073693,0.075255,0.075956,0.076435,0.077674,0.076923,...,0.004824,0.004681,0.004431,0.005047,0.004311,0.004521,0.004614,0.004367,0.004166,0.004335
264,0.087047,0.078448,1.000000,0.080070,0.081800,0.080350,0.076923,0.080634,0.082100,0.078448,...,0.004822,0.004674,0.004416,0.005035,0.004322,0.004543,0.004595,0.004365,0.004179,0.004333
356,0.081800,0.076435,0.080070,1.000000,0.076435,0.078187,0.075025,0.082403,0.077171,0.075956,...,0.004816,0.004671,0.004416,0.005038,0.004310,0.004526,0.004578,0.004365,0.004175,0.004339
412,0.080634,0.073693,0.081800,0.076435,1.000000,0.078711,0.075025,0.082403,0.078187,0.078448,...,0.004810,0.004702,0.004414,0.005034,0.004318,0.004530,0.004578,0.004367,0.004177,0.004349


## Step 4: Check your results by generating a list of the top 5 most similar customers for a specific CustomerID.

In [8]:
customer_id = 33;
top_5 = list(distances_df[customer_id].sort_values(ascending=False)[1:6].index)
top_5

[264, 3535, 3317, 2503, 3305]

## Step 5: From the data frame you created in Step 1, select the records for the list of similar CustomerIDs you obtained in Step 4.

In [9]:
items_of_similar_customers_df = quantity_df[quantity_df['CustomerID'].isin(top_5)].reset_index(drop=True)
items_of_similar_customers_df.head()

,CustomerID,ProductName,Quantity
0,264,Apricots - Halves,1
1,264,Apricots Fresh,1
2,264,Bacardi Breezer - Tropical,1
3,264,Bagel - Plain,1
4,264,Banana - Leaves,1


## Step 6: Aggregate those customer purchase records by ProductName, sum the Quantity field, and then rank them in descending order by quantity.

This will give you the total number of each product purchased by the 5 most similar customers to the customer you selected in order from most purchased to least.

In [10]:
items_df = items_of_similar_customers_df.groupby('ProductName').sum().sort_values(by='Quantity', ascending=False)['Quantity']
items_df.head()

ProductName
Butter - Unsalted                3
Wine - Ej Gallo Sierra Valley    3
Towels - Paper / Kraft           3
Soup - Campbells Bean Medley     3
Wine - Blue Nun Qualitatswein    3
Name: Quantity, dtype: int64

## Step 7: Filter the list for products that the chosen customer has not yet purchased and then recommend the top 5 products with the highest quantities that are left.

- Merge the ranked products data frame with the customer product matrix on the ProductName field.
- Filter for records where the chosen customer has not purchased the product.
- Show the top 5 results.

In [11]:
# Merge
merged = pd.merge(items_df, pivot_df[customer_id], on='ProductName').reset_index()
merged.columns = ['ProductName', 'Quantity', 'Purchased'] 

# Filter
filtered = merged[merged['Purchased'] == 0]

# Top 5
filtered.head()

,ProductName,Quantity,Purchased
0,Butter - Unsalted,3,0
1,Wine - Ej Gallo Sierra Valley,3,0
3,Soup - Campbells Bean Medley,3,0
4,Wine - Blue Nun Qualitatswein,3,0
6,Chicken - Soup Base,2,0


## Step 8: Now that we have generated product recommendations for a single user, put the pieces together and iterate over a list of all CustomerIDs.

- Create an empty dictionary that will hold the recommendations for all customers.
- Create a list of unique CustomerIDs to iterate over.
- Iterate over the customer list performing steps 4 through 7 for each and appending the results of each iteration to the dictionary you created.

In [12]:
# Empty dictionary
recommender_dict = {customer: [] for customer in set(data['CustomerID'])}

# Iteration
for customer_id in set(data['CustomerID']):
    # Step 4
    top_5 = list(distances_df[customer_id].sort_values(ascending=False)[1:6].index)
    # Step 5
    items_of_similar_customers_df = quantity_df[quantity_df['CustomerID'].isin(top_5)].reset_index(drop=True)
    # Step 6
    items_df = items_of_similar_customers_df.groupby('ProductName').sum().sort_values(by='Quantity', ascending=False)['Quantity']
    # Step 7
    merged = pd.merge(items_df, pivot_df[customer_id], on='ProductName').reset_index()
    merged.columns = ['ProductName', 'Quantity', 'Purchased'] 
    filtered = merged[merged['Purchased'] == 0]
    # Results
    recommender_dict[customer_id] = list(filtered['ProductName'].head())

##  Step 9: Store the results in a Pandas data frame. The data frame should have a column for Customer ID and then a column for each of the 5 product recommendations for each customer.

In [13]:
recommendations_df = pd.DataFrame.from_dict(recommender_dict, orient='index').reset_index()
recommendations_df.columns = ['CustomerID', '1', '2', '3', '4', '5']
recommendations_df.head(10)

,CustomerID,1,2,3,4,5
0,83973,Cocoa Butter,Veal - Slab Bacon,Bandage - Fexible 1x3,Ezy Change Mophandle,Chicken - Wieners
1,59399,"Shrimp - Baby, Warm Water",Knife Plastic - White,Towels - Paper / Kraft,Nantucket - Pomegranate Pear,Snapple - Iced Tea Peach
2,92168,Cup - Translucent 7 Oz Clear,"Veal - Brisket, Provimi,bnls",Snapple Lemon Tea,Spinach - Baby,Pop Shoppe Cream Soda
3,49159,"Pasta - Penne, Rigate, Dry",Cookies - Assorted,Puree - Mocha,Cheese - Parmesan Grated,Cheese - Victor Et Berthold
4,18441,Soupfoamcont12oz 112con,Anchovy Paste - 56 G Tube,Sprouts - Baby Pea Tendrils,"Lemonade - Natural, 591 Ml",Sausage - Breakfast
5,22536,Bacardi Breezer - Tropical,Cheese - Mozzarella,Cheese Cloth No 100,Bread - Italian Roll With Herbs,Bread - Hot Dog Buns
6,86028,Spinach - Baby,Pork - Hock And Feet Attached,Vinegar - Tarragon,Bread Foccacia Whole,Lettuce - Spring Mix
7,75791,Blueberries,Mussels - Cultivated,Vaccum Bag 10x13,Turkey - Oven Roast Breast,Bread - Calabrese Baguette
8,96272,Ecolab - Lime - A - Way 4/4 L,Bread Crumbs - Panko,Guinea Fowl,Sunflower Seed Raw,Bread - English Muffin
9,32785,Cookies - Assorted,Table Cloth 54x72 White,Juice - V8 Splash,Wine - Redchard Merritt,Puree - Passion Fruit


## Step 10: Change the distance metric used in Step 3 to something other than euclidean (correlation, cityblock, consine, jaccard, etc.). Regenerate the recommendations for all customers and note the differences.

In [14]:
# Step 3
distances = squareform(pdist(pivot_df.T, 'cosine'))
distances_df = pd.DataFrame(1 / (1 + distances), index=pivot_df.columns, columns=pivot_df.columns)

# Empty dictionary

recommender_dict_method_2 = {customer: [] for customer in set(data['CustomerID'])}

# Iteration
for customer_id in set(data['CustomerID']):
    # Step 4
    top_5 = list(distances_df[customer_id].sort_values(ascending=False)[1:6].index)
    # Step 5
    items_of_similar_customers_df = quantity_df[quantity_df['CustomerID'].isin(top_5)].reset_index(drop=True)
    # Step 6
    items_df = items_of_similar_customers_df.groupby('ProductName').sum().sort_values(by='Quantity', ascending=False)['Quantity']
    # Step 7
    merged = pd.merge(items_df, pivot_df[customer_id], on='ProductName').reset_index()
    merged.columns = ['ProductName', 'Quantity', 'Purchased'] 
    filtered = merged[merged['Purchased'] == 0]
    # Results
    recommender_dict_method_2[customer_id] = list(filtered['ProductName'].head())

In [15]:
recommender_dict[18680]

['Chocolate - Dark',
 'Pork - Kidney',
 'Wine - Gato Negro Cabernet',
 'Wanton Wrap',
 'Lettuce - Spring Mix']

In [16]:
recommender_dict_method_2[18680]

['Water - Mineral, Natural',
 'Onions - Vidalia',
 'Longos - Chicken Wings',
 'Tilapia - Fillets',
 'Mussels - Frozen']